In [2]:
%pip install PyYAML

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np 
import os 
import yaml 
from yaml.loader import SafeLoader 



In [6]:
# load yaml file
with open('data.yaml', mode = 'r') as f:
    data_yaml = yaml.load(f,Loader = SafeLoader)
labels = data_yaml['names']
print(labels)


['person', 'car', 'chair', 'bottle', 'pottedplant', 'bird', 'dog', 'sofa', 'bicycle', 'horse', 'boat', 'motorbike', 'cat', 'tvmonitor', 'cow', 'sheep', 'aeroplane', 'train', 'diningtable', 'bus']


In [16]:
# load YOLO model
yolo = cv2.dnn.readNetFromONNX('./data_images15/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# load image
img = cv2.imread('./street_image.jpg')
image = img.copy()
row, col, d = image.shape
print(row, col, d)
#show image
#cv2.imshow('image',image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
# get YOLO prediction from imageL
max_rc = max(row, col)
# convert image to square image (array)
input_image = np.zeros((max_rc, max_rc, 3),dtype=np.uint8)
input_image[0:row,0:col] = image
#cv2.imshow('input_image',input_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
# get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO), swapRB=True, crop=False)
yolo.setInput(blob)
preds = yolo.forward() # prediction from yolo
print(preds.shape)


956 1920 3
(1, 25200, 25)


#### The 25200 rows are 25200 bounding boxes
#### The 25 columns are 5 bounding box columns (CenterX, CenterY, W, H, Confidence) and
#### 20 Probability Scores for each class (person, bus, etc.)


In [30]:
# Non Maximum Suppression
# remove duplicates, non maximums and select only bounding boxes having good confidence scores (0.4) and high probability (0.25)
detections = preds[0] # 25200,25)
boxes = []
confidences = []
classes = []
# Draw the Bounding Box
image_w, image_h = input_image.shape[:2]
x_factor = image_w / INPUT_WH_YOLO
y_factor = image_h / INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]
    if confidence > 0.4: #select bounding box
        class_score = row[5:].max() # maximum probability of class object detected
        class_id = row[5:].argmax() # get index position of object

        if class_score > 0.25:
{% load             cx, cy, w, h = row[:4]
_tags %}            # construct bounding box 
            left = int((cx-0.5*w)* x_factor)
            top = int((cy-0.5*w)* y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)
            box = np.array([left,top,width,height])

            #append values into list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

# clean, remove duplicates
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#Non-Maximal suppression
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()



In [1]:
# Draw the Bounding Box
for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_confidence = round(confidences_np[ind] * 100,2)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    text = f'{class_name}: {bb_confidence}%'
    #print(text)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

cv2.imshow('Yolo Prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'index' is not defined